In [105]:
import urllib.request
import tempfile
import scrapbook
import uuid

In [106]:
url = "https://edc-oapi.dev.hub.eox.at/oapi/collections/S2L2A/coverage?f=GeoTIFF&subset=lat(16:16.001),lon(48:48.001),time(%222020-09-10T00:00Z%22:%222020-09-29T00:00Z%22)&rangeSubset=B04,B08"
with urllib.request.urlopen(url) as in_file:
    with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmp_file:
        tmp_file.write(in_file.read())
        tmp_file.flush()
        arr = xr.open_rasterio(tmp_file.name)


In [107]:
ds = arr.to_dataset("band").rename({1: "B04", 2: "B08"})
ndvi = (ds.B08 - ds.B04) / (ds.B04 + ds.B08)

In [108]:
!python3 -m pip -q install rioxarray
import rioxarray

output_filepath = f"/home/jovyan/s3/ndvi-{uuid.uuid4()}.tif"

ndvi.rio.to_raster(output_filepath)

scrapbook.glue("result-file", output_filepath)